In [1]:
import numpy as np
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time 
import pandas as pd
import json
import requests
from pyquery import PyQuery
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from requests.adapters import HTTPAdapter, Retry

In [2]:
def get_links(page_num):
    from selenium import webdriver
    import time
    from selenium.common.exceptions import StaleElementReferenceException

    service = Service(ChromeDriverManager().install())


    driver = webdriver.Chrome(service=service)
    driver.maximize_window()
    driver.implicitly_wait(10)
    driver.get("https://www.vishvasnews.com/english/")




    j = 0
    try:
        while j<page_num:
            nav_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "nav-link"))
                )

            load_more_link = nav_element.find_element(By.PARTIAL_LINK_TEXT, "Load More")
            #load_more_link.click()
            driver.execute_script("arguments[0].click();", load_more_link)
            time.sleep(20)
            j = j+1
    except Exception as e:
        print("Failed to click the 'Load More' link:", e) 

    content = driver.page_source
    soup = BeautifulSoup(content, "lxml")
    art_links = []
    data_links = soup.find_all("div", {"class": "imagebox"})
        # from each <a> element, extract the URL
    for i in data_links:
        ia = i.find('a')
        if ia:
            art_links.append(ia['href'])

    driver.quit()
    
    df = pd.DataFrame(article_links, columns = ["Links"])
    df.to_csv("Vishvas_News_english_links_june2020.csv")
    
    return

In [3]:
def get_content(dt):
    
    for url in dt["Links"]:

        session = requests.Session()
        retry = Retry(connect=3, backoff_factor=0.5)
        adapter = HTTPAdapter(max_retries=retry)
        session.mount('http://', adapter)
        session.mount('https://', adapter)

        response = session.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content of the page using BeautifulSoup
            soup = BeautifulSoup(response.text, 'html.parser')
            meta_tags = soup.find_all('meta')
            meta_features = {}
            for tag in meta_tags:
                if tag.get('property'):
                    meta_features[tag['property']] = tag.get('content')
            # title
            title.append(meta_features['og:title'])

            # Published_date
            Published_date.append(meta_features['article:published_time'])

            # top image
            if 'og:image' in meta_features:
                top_image.append(meta_features['og:image'])
            else:
                top_image.append(" ")

            # topic
            topic.append(meta_features['article:section'])

            # Content
            data = soup.find_all('div', {'class':"view-full"})
            if data:
                extracted_content = []
                for element in data:
                    for el in element.find_all(['p', 'blockquote']):
                        extracted_content.append(el.get_text())

                full_content = '\n'.join(extracted_content)

            Content.append(full_content)

            # Extract all links
            view_full_section = soup.find('div', class_='view-full')
            if view_full_section:
                view_full_links = view_full_section.find_all('a')
                linksss = []
                for link in view_full_links:
                    href = link.get('href')
                    if link:
                        linksss.append(link)

            links_in_text.append(linksss)

            # Claim
            claim_section = soup.find('h2', text='Claim')
            if claim_section:
                claim_content = ""
                for p_element in claim_section.find_next_siblings(['p', 'h2']):
                    if p_element.name == 'h2' and p_element.text == 'Investigation':
                        break
                    claim_content += p_element.get_text() + "\n"
            else:
                claim_content = " "

            claim.append(claim_content)


            # Investigation and conclusion
            investigation_section = soup.find('h2', text='Investigation')
            if investigation_section:
                investigation_content = ""
                for p_element in investigation_section.find_next_siblings(['p', 'h2']):
                    if p_element.name == 'h2' and p_element.text == 'Know The Truth… Spread Awareness':
                        break
                    investigation_content += p_element.get_text() + "\n"
            else:
                investigation_content = " "

            investigation.append(investigation_content)   


            # image links
            figure_elements = soup.find_all('figure', class_='wp-block-image')
            images = []
            for figure in figure_elements:
                img = figure.find('img', {'data-src': True})
                if img:
                    data_src = img['data-src']
                    images.append(data_src)

            image_links.append(images)

            # tags
            tags_ul = soup.find('ul', class_='tags')
            if tags_ul:
                tag_items = tags_ul.find_all('li')

                tagggs = [tag.text for tag in tag_items]
            tags.append(tagggs)

            # youtube links
            iframes = soup.find_all('iframe')
            y_links = []
            for iframe in iframes:
                if 'data-src' in iframe.attrs:
                    youtube_link = iframe['data-src']
                    y_links.append(youtube_link)

            youtube_links.append(y_links)

            linnnks.append(url)
    df1 = pd.DataFrame({'link':linnnks, 'title': title, 'publish_date': Published_date, 'content': Content, 'claim': claim, 'investigation': investigation, 'Links_in_text':links_in_text,'top image': top_image, 'image links':image_links, 'video':youtube_links, 'tags':tags, 'topic': topic})
    df1.to_csv('Vishvas_news_telugu_june2020.csv')
    return

In [12]:
if __name__ == "__main__":
    title = []
    Published_date = []
    Content = []
    claim = []
    investigation = []
    top_image = []
    links_in_text = []
    topic = []
    image_links = []
    youtube_links = []
    tags = []
    linnnks = []
    get_links(90)
    dataframe = pd.read_csv("Vishvas_News_telugu_links_june2020.csv")
    get_content(dataframe)